# Notebook 05 — Rolling Backtest & Monitoring

This notebook evaluates model stability under realistic deployment conditions
using time-aware rolling validation.

Key objectives:
- Avoid data leakage
- Evaluate temporal robustness
- Compare ROC-AUC, PR-AUC and calibration (Brier)


In [11]:
import os
import numpy as np
import pandas as pd
import duckdb

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss


## 1. Configuration

In [10]:
DB_PATH = '../lakehouse/analytics.duckdb'
TABLE = 'player_dataset_predictive_v2'

FEATURES = [
    "minutes_last_7d",
    "minutes_last_14d",
    "minutes_last_28d",
    "minutes_last_5_matches",
    "acwr",

    "minutes_std_last_5_matches",
    "minutes_std_last_10_matches",

    "delta_7d_14d",
    "delta_14d_28d",

    "acwr_change",

    "season_minutes_cum",
    "season_matches_played",
    "season_avg_minutes",

    "season_momentum_3v_season_avg"
]

TARGET = 'high_risk_next'

# Rolling configuration
TIME_BLOCK = 'W'
MIN_TRAIN_BLOCKS = 10
TEST_BLOCKS = 2
STEP_BLOCKS = 4
MAX_BLOCKS = 80


## 2. Load Data

In [12]:
con = duckdb.connect(DB_PATH, read_only=True)
df = con.execute(f'SELECT * FROM {TABLE}').fetchdf()
con.close()

df['match_date'] = pd.to_datetime(df['match_date'])
df = df.sort_values('match_date').reset_index(drop=True)

X = df[FEATURES]
y = df[TARGET].astype(int)

print('Shape:', df.shape)


Shape: (81158, 36)


## 3. Build Rolling Folds

In [13]:
def make_time_blocks(df, time_col, block='W'):
    return pd.to_datetime(df[time_col]).dt.to_period(block).astype(str)

blocks = make_time_blocks(df, 'match_date', TIME_BLOCK)
uniq = pd.Index(blocks.unique())[-MAX_BLOCKS:]

folds = []
start = MIN_TRAIN_BLOCKS

while start + TEST_BLOCKS <= len(uniq):
    tr = uniq[:start]
    te = uniq[start:start + TEST_BLOCKS]

    tr_idx = df.index[blocks.isin(tr)].values
    te_idx = df.index[blocks.isin(te)].values

    folds.append((tr_idx, te_idx))
    start += STEP_BLOCKS

print('Folds:', len(folds))


Folds: 18


## 4. Rolling Backtest

In [14]:
results = []

for fold_id, (tr_idx, te_idx) in enumerate(folds):

    X_train = X.iloc[tr_idx]
    y_train = y.iloc[tr_idx]
    X_test  = X.iloc[te_idx]
    y_test  = y.iloc[te_idx]

    logit = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ("model", LogisticRegression(
            max_iter=2000,
            C=0.5, #moderate regularization
            solver="lbfgs"
        )),
    ])

    logit.fit(X_train, y_train)
    p = logit.predict_proba(X_test)[:, 1]

    results.append({
        'model': 'logit',
        'fold': fold_id,
        'roc_auc_test': roc_auc_score(y_test, p),
        'pr_auc_test': average_precision_score(y_test, p),
        'brier_test': brier_score_loss(y_test, p),
    })

pd.DataFrame(results).groupby('model').agg(['mean','std'])


fold           roc_auc_test           pr_auc_test           brier_test  \
      mean       std         mean       std        mean       std       mean   
model                                                                          
logit  8.5  5.338539     0.740365  0.174794    0.663282  0.262685    0.21349   

                 
            std  
model            
logit  0.093895

## Model Robustness Experiments

To assess structural stability and avoid overfitting-driven performance inflation,
a series of controlled experiments were conducted under rolling time validation.

### 1. Regularization Test
- Logistic regression with C=0.5 (stronger L2 regularization).
- Result: negligible change in mean ROC-AUC.
- Temporal variance remained similar.
- Conclusion: instability is not driven by excessive coefficient magnitude.

### 2. Feature Pruning (Ratios Removed)
Removed:
- ratio_7d_14d
- ratio_14d_28d
- minutes_last_3_matches

Result:
- Mean ROC-AUC decreased.
- Variance increased.
- Calibration deteriorated.

Conclusion:
Ratio-based workload interactions contribute meaningful predictive signal.
Temporal variability is likely due to non-stationary competitive dynamics
rather than classical overfitting.

### Final Assessment

The model demonstrates:
- Solid average ranking power (ROC ≈ 0.76)
- Acceptable calibration (Brier ≈ 0.20)
- Moderate temporal variability

Observed instability reflects real-world workload dynamics rather than model mis-specification.